# Item-item collaborative filtering

In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy 

from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)
from pyspark.ml.recommendation import ALS
from pyspark.sql import functions as F

In [6]:
data = pd.read_csv("/home/romain/Documents/PhD/logisticDPP/UKRetailer/data.csv",encoding="latin1")

nItem = len(set(data['StockCode']))
pivotTable = pd.DataFrame({'StockCode':list(set(data['StockCode'])),
                           'itemid':list(range(nItem))})

data = pd.merge(data,pivotTable)
data = data.groupby(['CustomerID','InvoiceDate'])['itemid'].apply(list).reset_index()
del data['CustomerID']
del data['InvoiceDate']
data = data.rename(columns={'itemid':'itemSet'})

data['itemSet'] = list(map(lambda x: list(set(x)),data['itemSet']))
data['setSize'] = list(map(len,data['itemSet']))

data = data.loc[data['setSize']<100,]
data = data.loc[data['setSize']>1]

sets = list(map(lambda x: list(map(lambda y: int(y),x)),data['itemSet']))
size = list(map(len,sets))
nUsers = len(sets)
users = np.repeat(range(nUsers),size)

flatsets = [int(item) for sublist in sets for item in sublist]
df = pd.DataFrame({'user':users,'item':flatsets,'rating':1.0})

In [3]:
def random_mask(x):
    result = np.zeros_like(x)
    if len(x)>1:
        result[np.random.choice(len(x))] = 1
    return result

threshold   = 0.7

In [4]:
def recommendForBasket(basket,V):
    sim = np.zeros(len(V))
    for item in basket:
        tmp = V[item,:].dot(V.T)
        sim += tmp/len(basket)
    sim[basket] = np.min(sim)-10
    return sim

In [16]:
sc.setCheckpointDir("checkpoints")
for numTraits in [5,10,20,30,50]:
    print("Start numTraits=",numTraits)
    
    nRuns       = 3
    MPR         = []
    Ks          = [5,10,20]
    P           = dict.fromkeys(Ks)
    for K in Ks:
        P[K] = []

    for run in range(nRuns):
        print("run number",run+1,"-",numTraits)
        np.random.seed(123*run)

        testUsers = list(np.random.choice(range(nUsers),size=int((1-threshold)*nUsers),replace=False))
        trainingUsers = list(set(range(nUsers))-set(testUsers))
        trainingData = df.loc[trainingUsers]
        trainingData.index = range(len(trainingData))

        testData = df.loc[testUsers]
        testData.index = range(len(testData))

        mask = testData.groupby(['user'])['user'].transform(random_mask).astype(bool)
        not_mask = list(map(lambda x: not(x),mask))
        
        testUsersBasket = testData.loc[not_mask]
        #trainingData = pd.concat([trainingData,testData.loc[not_mask]])
        testData = testData.loc[mask]

        sparkInput = sqlCtx.createDataFrame(trainingData)
        als = ALS(rank=numTraits,regParam=0.1,userCol='user',itemCol='item',ratingCol='rating',implicitPrefs=True)
        mod = als.fit(sparkInput) 

        # get item latent factors
        V = mod.itemFactors.orderBy("id")
        V_index = V.select('id').toPandas()
        V = V.select('features')

        for k in range(numTraits):
            V = V.withColumn('factor'+str(k),V.features[k])

        V = V.drop('features')
        V = V.toPandas()
        V.index = V_index['id']
        unknowns = list(set(range(nItem))-set(V_index['id']))
        for unknown in unknowns:
            V.loc[unknown] = 0
        V = V.sort_index()
        V = np.array(V)

        percentileRank = []
        precisionAt5 = 0
        precisionAt10 = 0
        precisionAt20 = 0

        for user in testUsers:
            if len(testData.loc[testData['user']==user,'item'])>0:
                basket = list(testUsersBasket.loc[testUsersBasket['user']==user,'item'])
                true_target = list(testData.loc[testData['user']==user,'item'])[0]
                subY = recommendForBasket(basket,V)
                y0 = subY[true_target]
                rank = np.sum(subY>y0)
                percentileRank.append(1-rank/nItem)
                top5Target = np.argsort(subY)[-5:]
                top10Target = np.argsort(subY)[-10:]
                top20Target = np.argsort(subY)[-20:]
                if true_target in top5Target:
                    precisionAt5 += 1
                if true_target in top10Target:
                    precisionAt10 += 1
                if true_target in top20Target:
                    precisionAt20 += 1

        MPR.append(100*np.mean(percentileRank))
        P[5].append(100*precisionAt5/len(percentileRank))
        P[10].append(100*precisionAt10/len(percentileRank))
        P[20].append(100*precisionAt20/len(percentileRank))
    
    print("\n")
    print("*"*20)
    print("num latent factors:",numTraits)
    print("MPR=",np.mean(MPR))
    for K in Ks:
        print("Precision @"+str(K)+"=",np.mean(P[K]))
    print("*"*20)

Start numTraits= 5
run number 1 - 5
run number 2 - 5
run number 3 - 5


********************
num latent factors: 5
MPR= 85.43399263530796
Precision @5= 7.56286859430314
Precision @10= 11.407594734814777
Precision @20= 15.531797432175233
********************
Start numTraits= 10
run number 1 - 10
run number 2 - 10
run number 3 - 10


********************
num latent factors: 10
MPR= 85.1250717576973
Precision @5= 6.858143768975299
Precision @10= 10.95385081740215
Precision @20= 17.238430155203805
********************
Start numTraits= 20
run number 1 - 20
run number 2 - 20
run number 3 - 20


********************
num latent factors: 20
MPR= 81.66558023231407
Precision @5= 8.068300116130509
Precision @10= 11.774017717971061
Precision @20= 17.254324713151693
********************
Start numTraits= 30
run number 1 - 30
run number 2 - 30
run number 3 - 30


********************
num latent factors: 30
MPR= 81.99927177148557
Precision @5= 8.686254754185649
Precision @10= 11.856034958774936
Precisi

In [15]:
basket

[1819, 2680, 3726, 611]